## DATA LOADING 

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# THIS TAKES A WHILE TO RUN 
df_train = pd.read_csv("train.csv")
df_train.head()

In [ ]:
len(df_train)

In [ ]:
# Checking for duplicate data 
print('Number of duplicate row:')
df_train.bidid.duplicated().sum()


## DATA EXPLORATION

In [ ]:
df_train.info()

In [ ]:
# Many row with payprice = 0 , why payprice = 0 ? 
print('Many rows have payprice = 0')
len(df_train[df_train['payprice'] == 0])

# Find out which slotid has zero payprice -> slotid QQlive has majority of zero payprice 
print()
print('The slotid have payprice = 0 are:')
df_train[df_train['payprice'] == 0].groupby(['slotid']).size()


In [ ]:
# Look at click and non click for each advertiser

df_click = df_train.groupby(['advertiser','click']).size().reset_index(name ='click_or_no_click')
df_click

In [ ]:
# Nummber of click 
print('NUMBER OF IMPRESSIONS:', len(df_train))
print()

click = df_train.groupby(['click']).size()
print('NUMBERS OF IMPRESSION THAT HAVE CLICKS (= 1) AND NOT HAVING CLICKS (= 0):')

print(click)

print()
print('STATISTIC FOR IMPRESSIONS THAT HAD NO CLICK:')
df_train[df_train['click'] == 0].describe()

In [ ]:
# Compare the bidprice and pay price for impression that had click vs not having clicks 

print('STATISTIC FOR IMPRESSIONS THAT HAD CLICKS:')
df_train[df_train['click'] == 1].describe()


## 2. STATISTIC

### 2.1 GENERAL STATISTIC FOR THE WHOLE DATA SET

In [ ]:
# Drop all the columns that are not needed for now 
df_stat = df_train.drop(columns= ['weekday','hour','userid','useragent','IP','region','city','adexchange','domain','url','urlid','slotid','slotwidth','slotheight','slotvisibility','slotformat','creative','keypage','usertag'])

# Add col for pay price for one imp 
payprice_per_imp = df_stat['payprice'] / 1000
df_stat['payprice_per_imp'] = payprice_per_imp.values 
df_stat.head()

#### Number of advertisers and their bids (aka number of impressions)

In [ ]:
print('NUMBER OF IMPRESSIONS:', len(df_stat))
print()

print('STATISTIC FOR CPM:')
print(df_stat['payprice'].describe())

print()
print('STATISTIC FOR PAY PER IMPRESSION:', )
print(df_stat['payprice_per_imp'].describe())

print()
print('STATISTIC FOR SLOTPRICE:')
print(df_stat['slotprice'].describe())

print()
print('STATISTIC FOR BIDPRICE:')
print(df_stat['bidprice'].describe())

print()
print('STATISTIC FOR BIDPRICE:')
print(df_stat['payprice'].describe())



In [ ]:
# I got the formulas from here
#http://www.usefullytips.com/2016/06/how-to-calculate-cpc-cpa-cpm-ecpc-ecpm.html
# Can you please check if these are correct 
 

# eCPM Total Earnings/Impressions x 1,000. -> WE DONT HAVE TOTAL EARNING 
# https://www.techwalla.com/articles/ecpm-vs-cpm

# CPM cost per mille 
CPM = df_stat['payprice'].sum()/len(df_stat)
print('AVERAGE CPM:', CPM) 


# Cost Per Click = cost to an advertiser / number of clicks
CPC = df_stat['payprice'].sum()/df_stat['click'].sum()
print('AVERAGE CPC:', CPC)  

# Click Through Rate = (Number of clicks / number of impressions) x 100
CTR = (len(df_stat)/len(df_stat))*100
print('AVERAGE CTR:', CTR)  



### 2.2 STATISTIC ANALYSIS FOR EACH ADVERTISERS

In [ ]:
# Calculate number of bids(aka number od imps) for each advertisers

bid_counts = df_stat.groupby(['advertiser']).agg({'bidid':'size'}).rename(columns={'bidid':'number_of_bids'}) 

In [ ]:
# Statistic for each advertiser: average bidprice, pay price... 
#df_advertiser_stat.reset_index(name = 'advertisers')

advertisers_grouped = df_stat.groupby(['advertiser']).mean()
advertisers_grouped

In [ ]:
print('NUMBER OF ADVERTISERS:', len(advertisers_grouped), 'ADVERTISERS')

In [ ]:
# Adding column 'number of bids' 
advertiser_stat = pd.concat([advertisers_grouped, bid_counts], axis=1)
advertiser_stat

In [ ]:
# Plots:
# Compare average click rate and pay price for each advertiser 


fig = plt.figure(figsize=(13, 7))
fig.subplots_adjust(wspace = 0.5 )

plt.subplot(221)
ax11 = advertiser_stat['payprice'].plot(kind = 'bar', color ='black')
plt.ylabel('Average Pay Price (black)')
ax21 = ax11.twinx()
ax21.plot(ax11.get_xticks(), advertiser_stat['click'], marker = 'o', color = 'red')
[tl.set_color('r') for tl in ax21.get_yticklabels()]
plt.ylabel('Click Rate (red)',color = 'red')
ax11.set_title('Click Rate on Average Pay Price')


plt.subplot(222)
ax = advertiser_stat['number_of_bids'].plot(kind ='bar', color = 'gray')
plt.ylabel('Number of Bids (gray)')
ax2 = ax.twinx()
ax2.plot(ax.get_xticks(), advertiser_stat['click'], marker = 'o', color = 'red')
[tl.set_color('red') for tl in ax2.get_yticklabels()]
plt.ylabel('Click Rate (red)', color = 'red')
ax2.set_title('Click Rate on Number of Bids (aka number of imps)')

plt.show()

In [ ]:
# Compare Bid Price and PayPrice 

ax0 = advertiser_stat['bidprice'].plot(kind = 'bar', color = 'black')

ax1 = advertiser_stat['payprice'].plot(kind = 'bar', color = 'red')

ax2 = advertiser_stat['slotprice'].plot(kind = 'bar', color = 'gray')
plt.ylabel('Average Slot Price, Bid Price, Pay Price')

ax0.legend(loc='upper center', bbox_to_anchor=(1.2, 1),  shadow=True, ncol=1 )
plt.show()

#ax2 = ax0.twinx()
#ax2.plot(ax0.get_xticks(), df_advertiser_stat['click'], marker = 'o', color = 'red')
#[tl.set_color('r') for tl in ax2.get_yticklabels()]
#plt.ylabel('Click Rate ()', color = 'red')
 
plt.show()

In [ ]:
# Sum values for calculate CPM, CPC for each advertiser
advertisers_sum = df_stat.groupby(['advertiser']).sum()
advertisers_sum = pd.concat([advertisers_grouped, bid_counts], axis=1)
advertisers_sum

In [ ]:

# Calculate CPC  
CPC = advertisers_sum['payprice']/advertisers_sum['click']
print('CPC:', CPC)
print()

# CPM = (total spent / Impressions delivered )
CPM = (advertisers_sum['payprice']/advertisers_sum['number_of_bids'])
print('CPM:', CPM)

# NOT SURE 
# eCPC = (Total spent or revence / clicks )
